In [20]:
print("Name: Sukrutha D")
print("SRN: PES2UG23CS622")

Name: Sukrutha D
SRN: PES2UG23CS622


# **Unit 2 Assignment: Building a Mixture of Experts (MoE) Router**

In [1]:
!pip install groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 2.9 MB/s eta 0:00:00


In [2]:
with open(".env", "w") as f:
    f.write("GROQ_API_KEY=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")

In [9]:
import os
from dotenv import load_dotenv
from groq import Groq
load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

BASE_MODEL = "llama-3.1-8b-instant"

print("Groq client initialized successfully!")

Groq client initialized successfully!


In [10]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": """
You are a Technical Support Expert.
Be precise, logical, and code-focused.
If debugging, explain the root cause and provide corrected code.
Avoid unnecessary politeness. Be direct and structured.
"""
    },
    "billing": {
        "system_prompt": """
You are a Billing Support Expert.
Be empathetic, polite, and policy-driven.
Clearly explain refund rules, subscription policies, and next steps.
Always reassure the customer.
"""
    },
    "general": {
        "system_prompt": """
You are a General Customer Support Assistant.
Handle casual conversation and general queries politely and helpfully.
"""
    }
}

In [11]:
def route_prompt(user_input: str) -> str:
    """
    Classifies query into:
    technical, billing, general
    Returns ONLY category name.
    """

    routing_prompt = f"""
Classify this text into one of these categories:
[technical, billing, general]

Return ONLY the category word.

Text:
{user_input}
"""

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a strict text classifier."},
            {"role": "user", "content": routing_prompt}
        ]
    )

    return response.choices[0].message.content.strip().lower()

**Orchestrator Function**

In [12]:
def process_request(user_input: str) -> str:
    category = route_prompt(user_input)
    print(f"[Router Selected]: {category}")

    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content

**Testing Technical Query **

In [13]:
query = "My python script is throwing an IndexError on line 5."
response = process_request(query)

print("\nExpert Response:\n")
print(response)

[Router Selected]: technical

Expert Response:

To troubleshoot the issue, I'll need more information. Please provide the following:

1. The line numbers of your Python script (not line numbers starting from 1, but actual line numbers).
2. The code snippet where the error occurs.
3. The full error message, including the stack trace.

Once I have this information, I can help you identify the root cause and provide the necessary corrections.

If you cannot provide the exact line numbers, please paste the entire code snippet.


**Testing Billing Query**

In [14]:
query = "I was charged twice for my subscription this month."
response = process_request(query)

print("\nExpert Response:\n")
print(response)

[Router Selected]: billing

Expert Response:

I'm so sorry to hear that you've been charged twice for your subscription. I'm here to help and make things right. Can you please confirm your account information, including your name and the date of the charges, so I can look into this further?

In terms of our refund policy, I'd like to clarify that we offer a refund for duplicate or incorrect charges. If we find that the charges were indeed errors on our part, we'll process a refund as soon as possible.

However, if the charges were due to a change in your subscription plan or billing cycle, our standard refund policy may apply. This policy states that we don't offer refunds for subscription charges, but we do offer a prorated refund for prepaid plans.

Let's review your account and determine the best course of action. In the meantime, I can offer you a temporary credit on your account, which will be applied to your next payment. This way, you won't be charged again for the duplicate amo

# **Bonus Challenge**

In [16]:
def get_bitcoin_price():
    return "$63,450 (Mock Price)"

In [17]:
def route_prompt(user_input: str) -> str:

    routing_prompt = f"""
Classify this text into one of these categories:
[technical, billing, general, crypto]

Return ONLY the category word.

Text:
{user_input}
"""

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a strict text classifier."},
            {"role": "user", "content": routing_prompt}
        ]
    )

    return response.choices[0].message.content.strip().lower()

In [18]:
def process_request(user_input: str) -> str:
    category = route_prompt(user_input)
    print(f"[Router Selected]: {category}")

    if category == "crypto":
        return f"The current price of Bitcoin is {get_bitcoin_price()}"

    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content

In [19]:
query = "What is the current price of Bitcoin?"
response = process_request(query)

print("\nExpert Response:\n")
print(response)

[Router Selected]: crypto

Expert Response:

The current price of Bitcoin is $63,450 (Mock Price)
